In [1]:
from ppq import *
from ppq.api import *
import torch
import cfg
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/




In [2]:
# 消除冗余算子
# import onnx
# from onnxsim import simplify

# onnx_file = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32-end-11.onnx"
# onnx_model = onnx.load(onnx_file) 
# model_simp, check = simplify(onnx_model)   #对onnx模型进行简化，消除冗余算子        
# assert check, "Simplified ONNX model could not be validated"
# onnx.save(model_simp, onnx_file)

In [3]:
# 升级模型
import onnx
from onnx import version_converter, helper

# # Preprocessing: load the model to be converted.
# model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx"
# original_model = onnx.load(model_path)

# # A full list of supported adapters can be found here:
# # https://github.com/onnx/onnx/blob/main/onnx/version_converter.py#L21
# # Apply the version conversion on the original model
# converted_model = version_converter.convert_version(original_model, 11)

# onnx.save(converted_model, "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx")

In [2]:
model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx"
# model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/end2end-12.onnx"

from dataset import build_dataset
import torch
import cfg

model_name = "Retinanet"
input_size = cfg.MODELS[model_name]["INPUT_SHAPE"]
# 获取校准数据集
_,calib_dataloader =  build_dataset(ann_file=cfg.ANN_PATH,data_root=cfg.DATA_ROOT,
        input_size=input_size,batch_size=cfg.CALIBRATION_BATCH_SIZE)

# calib_dataloader = [torch.rand(size=input_size) for _ in range(512)]


config = cfg.PLATFORM_CONFIGS["TRT"]
config["QuanSetting"].dispatcher = "conservative"

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!


In [3]:
ppq_quant_ir = quantize_onnx_model(
    onnx_import_file=model_path, calib_dataloader=calib_dataloader, calib_steps=8 // cfg.CALIBRATION_BATCH_SIZE, 
    setting=config["QuanSetting"],input_shape=input_size, collate_fn=lambda x: x["img"][0].to(cfg.DEVICE), 
    platform=config["QuantPlatform"], do_quantize=True)

[18:29:18] PPQ Quantization Config Refine Pass Running ... Finished.
[18:29:18] PPQ Quantization Fusion Pass Running ...        Finished.
[18:29:19] PPQ Quantize Simplify Pass Running ...          Finished.
[18:29:19] PPQ Parameter Quantization Pass Running ...     

Calibration Progress(Phase 1):   0%|          | 0/512 [00:00<?, ?it/s]

Finished.
[18:29:19] PPQ Runtime Calibration Pass Running ...        

Calibration Progress(Phase 1):   0%|          | 1/512 [00:07<1:04:11,  7.54s/it]


KeyboardInterrupt: 

In [31]:
from inference import ppq_inference
dataloader = [next(iter(calib_dataloader))]
outputs = ppq_inference(dataloader=dataloader,ppq_ir = ppq_quant_ir,device = "cuda")

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Error happens when dealing with operation Conv_37(TargetPlatform.TRT_INT8) - inputs:['input', 'onnx::Conv_1245', 'onnx::Conv_1246'], outputs:['input.3']

In [28]:
def post_process(model_type,outputs,class_num):
    """
    将任意模型的输出转换为常规输出
    [N,result]  N为样本数
    result: [C,[M,5]] 其中C为类别,M为bbox数目，5为左上角、右下角坐标与置信分数。
    并且根据尺度进行复原
    """
    if model_type == "Retinanet":
        print("deal with outputs in Retinanet type")
        results = []
        for output in outputs:
            img_scale = output["scale_factor"] 
            labels,bboxs = output["output"]
            result = [[] for _ in range(class_num)]
            if len(labels.shape) > 2:
                # labels 和 bbox的输出顺序发生错换，需要调整
                bboxs,labels = labels,bboxs
                
            bboxs,labels = bboxs[0],labels[0]
            for i,label in enumerate(labels):
                bboxs[i][:4] = bboxs[i][:4] / img_scale  # 进行bbox的尺寸复原
                result[label].append(bboxs[i])
            results.append(result)
        return results
    elif model_type == "fasterrcnn":
        # TODO
        pass 

In [29]:
results = post_process(model_name,outputs,80)

deal with outputs in Retinanet type


In [30]:
results


[[[tensor([390.2247,  69.4758, 496.9633, 349.0060,   0.9082]),
   tensor([  0.0000, 263.6881,  62.3369, 308.3753,   0.4631]),
   tensor([7.3020e-01, 2.2863e+02, 6.4735e+01, 3.0202e+02, 1.8964e-01])],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [tensor([4.2743e+02, 1.1253e+02, 4.6965e+02, 1.8548e+02, 2.4914e-01]),
   tensor([4.0464e+02, 1.1265e+02, 4.6886e+02, 2.0723e+02, 2.3408e-01])],
  [],
  [tensor([4.1010e+02, 1.1412e+02, 4.6897e+02, 2.1819e+02, 3.2612e-01]),
   tensor([4.2743e+02, 1.1253e+02, 4.6965e+02, 1.8548e+02, 2.6462e-01]),
   tensor([3.9832e+02, 1.1009e+02, 4.7333e+02, 2.8062e+02, 2.5229e-01]),
   tensor([3.5515e+02, 1.9895e+02, 3.9580e+02, 2.9157e+02, 1.8748e-01])],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [tensor([130.5934, 104.4944, 141.5827, 133.0618,   0.2439]),
   tensor([3.8818e+02, 2.9966e+01, 4.1541e+02, 1.3671e+02, 2.1871e-01]),
   tenso

In [14]:
model_name = "Retinanet"
export_ppq_graph(   graph = ppq_quant_ir,
                    platform=TargetPlatform.ONNXRUNTIME,
                    graph_save_to=f'{os.path.join(config["OutputPath"], model_name)}-ORT-END-INT8.onnx')

In [7]:
x = next(iter(calib_dataloader))

In [8]:
x["img"][0].shape

torch.Size([2, 3, 800, 1216])

In [4]:
calib_input_size

(512, 3, 800, 1216)

In [ ]:
"""
op Less need opset (12,16)
op TopK need opset (1,12)
Upsample not supported
"""